## 수정사항
#### 2021.06.04
* Gripper 자유도 45도로 제한됨에 따라 파지점 방향 수정, 바닥 파지면은 삭제

## set running directory

In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [2]:
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir


DATA_PATH = os.path.join(os.environ['RNB_PLANNING_DIR'], "data")
try_mkdir(DATA_PATH)

TEST_DATA_PATH = os.path.join(DATA_PATH, "test-hoof")
try_mkdir(TEST_DATA_PATH)

In [3]:
VISUALIZE = True
TIMEOUT_REACH = 30
TIMEOUT_RETRIEVE = 30
TIMEOUT_SELF = 3
TOOL_RPY = (-np.pi/2,0,0)
SHOW_PERIOD = 0.01

ROBOT_TYPE = RobotType.panda

if ROBOT_TYPE == RobotType.indy7:
    ROBOT_NAME = "indy0"
    TOOL_LINK = "indy0_tcp"
    TOOL_XYZ = (0,0,0.14)
    GRIP_DEPTH = 0.05
    HOME_POSE = (0,0,0,0,0,0)
elif ROBOT_TYPE == RobotType.panda:
    ROBOT_NAME = "panda0"
    TOOL_LINK = "panda0_hand"
    TOOL_XYZ = (0,0,0.112)
    GRIP_DEPTH = 0.03
    HOME_POSE = (0,-0.3,0,-0.5,0,2.5,0)
else:
    raise(NotImplementedError("Unknown robot type"))
    
    
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, ROBOT_TYPE.name)
try_mkdir(ROBOT_DATA_ROOT)

## init combined robot config

In [4]:

from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, ROBOT_TYPE, None,
                INDY_IP)]
              , connection_list=[False])

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# # deprecated: s_builder.reset_reference_coord(ref_name="floor")

connection command:
panda0: False


## get ghnd with detected robot config

In [5]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {ROBOT_NAME: ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob, start_rviz=VISUALIZE)
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)


Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]


In [6]:
from pkg.utils.joint_utils import get_tf
shoulder_link = gscene.urdf_content.joint_map[gscene.joint_names[1]].child
shoulder_height = get_tf(shoulder_link, HOME_DICT, gscene.urdf_content)[2,3]

## add environment

In [7]:
from pkg.geometry.geometry import *
gtems_robot = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

Please create a subscriber to the marker


## init planning scene

In [8]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

## Register binders

In [9]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [10]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name=TOOL_LINK, 
                 dims=(0.01,)*3, center=TOOL_XYZ, rpy=TOOL_RPY, color=(1,0,0,1), display=True, collision=False, fixed=True)
gripper = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0))

## planner

In [11]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker


gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
# lcheck = LatticedChecker(pscene, gcheck)

g_checker = MoveitPlanner(pscene, motion_filters = [gcheck])
gr_checker = MoveitPlanner(pscene, motion_filters = [gcheck, rcheck])
# grl_checker = MoveitPlanner(pscene, motion_filters = [gcheck, rcheck, lcheck])

## ui

In [12]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


# Object Classes

In [13]:
from pkg.planning.filtering.lattice_model import *
from pkg.planning.filtering.lattice_model.scene_building import *
        
##
# @class PlanePan
# @brief Obstacles on the workplane
class PlaneHoof(PlaneObject):
    def __init__(self, gscene, name, workplane, GRIP_DEPTH, CLEARANCE, XYZ_LOC=None, **kwargs):
        self.GRIP_DEPTH = GRIP_DEPTH
        self.DIM_MIN = (0.02, GRIP_DEPTH, GRIP_DEPTH)
        DIM = np.random.uniform(self.DIM_MIN, self.DIM_MAX)
#         DIM[2] = GRIP_DEPTH
        PlaneObject.__init__(self, gscene=gscene, name=name, workplane=workplane, 
                             GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE, DIM=DIM, **kwargs)
        HOOF_LEN = 0.05
        gparent = self.geometry
        gscene.create_safe(GEOTYPE.BOX, gparent.name+"_cn_a", link_name=gparent.link_name, 
                           dims=(DIM[0], HOOF_LEN, DIM[2]), center=(-HOOF_LEN/2/np.sqrt(2),DIM[1]/2+HOOF_LEN/2/np.sqrt(2),0), rpy=(0,0,np.pi/4), 
                           color=gparent.color, display=True, collision=True, fixed=False,
                           parent=gparent.name)
        gscene.create_safe(GEOTYPE.BOX, gparent.name+"_cn_b", link_name=gparent.link_name, 
                           dims=(DIM[0], HOOF_LEN, DIM[2]), center=(-HOOF_LEN/2-HOOF_LEN/np.sqrt(2),DIM[1]/2+HOOF_LEN/np.sqrt(2),0), 
                           rpy=(0,0,np.pi/2), 
                           color=gparent.color, display=True, collision=True, fixed=False,
                           parent=gparent.name)
        gscene.create_safe(GEOTYPE.BOX, gparent.name+"_cn_c", link_name=gparent.link_name, 
                           dims=(DIM[0], HOOF_LEN, DIM[2]), center=(-HOOF_LEN-HOOF_LEN/np.sqrt(2)-HOOF_LEN/np.sqrt(2)/2,DIM[1]/2+HOOF_LEN/2/np.sqrt(2),0), 
                           rpy=(0,0,np.pi*3/4), 
                           color=gparent.color, display=True, collision=True, fixed=False,
                           parent=gparent.name)
        gscene.create_safe(GEOTYPE.BOX, gparent.name+"_cn_d", link_name=gparent.link_name, 
                           dims=(DIM[0], DIM[1], DIM[2]), center=(-HOOF_LEN-HOOF_LEN/np.sqrt(2)-HOOF_LEN/np.sqrt(2),0,0), 
                           rpy=(0,0,0), 
                           color=gparent.color, display=True, collision=True, fixed=False,
                           parent=gparent.name)


def add_object(pscene, obj, HANDLE_THICKNESS=1e-6, HANDLE_COLOR = (1,0,0,0.3)):
    gscene = pscene.gscene
    GRIP_DEPTH = obj.GRIP_DEPTH
    handles = []
    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_tp_a", link_name="base_link", 
                       dims=(obj.DIM[1], GRIP_DEPTH, HANDLE_THICKNESS), center=(0,0,obj.DIM[2]/2-GRIP_DEPTH/2), rpy=(np.pi/2,0,np.pi/2), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_tp_b", link_name="base_link", 
                       dims=(obj.DIM[1], GRIP_DEPTH, HANDLE_THICKNESS), center=(0,0,obj.DIM[2]/2-GRIP_DEPTH/2), rpy=(np.pi/2,0,-np.pi/2), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_bk_a", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,-obj.DIM[1]/2+GRIP_DEPTH/2,0), rpy=(-np.pi,-np.pi/2,0), 
                       color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                   parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_bk_b", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,-obj.DIM[1]/2+GRIP_DEPTH/2,0), rpy=(-np.pi,+np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    action_points_dict = {"placement": PlacePoint("placement", obj.geometry, [0,0,-obj.DIM[2]/2], [0,0,0])}
    action_points_dict.update({handle.name: Grasp2Point(handle.name, handle, None, (0,0,0)) for handle in handles})
    obj_pscene = pscene.create_subject(oname=obj.name, gname=obj.name, _type=CustomObject, 
                                 action_points_dict=action_points_dict)
    return obj_pscene, handles
        
Nmax_wall, Nmax_pole, Nmax_bar = 4, 8, 8

Nmax_obj = 3

N_max_sample = 100
N_print = 5
N_sample_single_obj = 3

   Use a production WSGI server instead.
 * Debug mode: off


# Load and verify

In [14]:
N_MAX = 1000


gtimer = GlobalTimer.instance()
gtimer.reset(stack=True)
DATASET_LIST = sorted(os.listdir(ROBOT_DATA_ROOT))
N_tot = 0
N_suc = 0
labels = []
g_results = []
gr_results = []
grl_results = []
print("DATASET_LIST: {}".format(DATASET_LIST))
gtimer.tic("full_loop")
DATASET_LIST = iter(DATASET_LIST)

DATASET_LIST: ['20210611-142747', '20210611-143200', '20210611-143507', '20210611-143922', '20210611-144222', '20210611-144701', '20210611-145001', '20210611-145428', '20210611-145955', '20210611-150331', '20210613-150300']


In [15]:
DATASET_DIR = next(DATASET_LIST)

DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DATASET_DIR)
print("DATASET_PATH: {}".format(DATASET_PATH))
DATA_LIST = sorted(os.listdir(DATASET_PATH))
print("DATA_LIST: {}".format(DATA_LIST))
DATA_LIST = iter(DATA_LIST)

DATASET_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-142747
DATA_LIST: ['00001.pkl', '00002.pkl', '00003.pkl', '00004.pkl', '00005.pkl', '00006.pkl', '00007.pkl', '00008.pkl', '00009.pkl', '00010.pkl', '00011.pkl', '00012.pkl', '00013.pkl', '00014.pkl', '00015.pkl', '00016.pkl', '00017.pkl', '00018.pkl', '00019.pkl', '00020.pkl']


In [27]:
DATA_FILE = next(DATA_LIST)

In [28]:

DATA_PATH = os.path.join(DATASET_PATH, DATA_FILE)
print("DATA_PATH: {}".format(DATA_PATH))
data = load_pickle(DATA_PATH)
gtem_args = data["gtem_args"]
obj_args = data["obj_args"]
samples = data["samples"]
gtem_remove = []
for gtem in gscene:
    if gtem.link_name == "base_link" and gtem.parent is None:
        gtem_remove.append(gtem)
for gtem in gtem_remove:
    gscene.remove(gtem)

gid_list = np.arange(len(gtem_args)).tolist()
for gidx in gid_list:
    args = gtem_args[gidx]
    if args['parent'] is not None:
        if args['parent'] not in gscene.NAME_DICT:
            gid_list.append(gidx)
            continue
    gscene.create_safe(**args)

pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-142747/00006.pkl


In [14]:

            
        for i_s, sample in enumerate(samples):
            obj_name, from_state, to_state, redundancy_dict, \
                success_kin_, success_reach_, success_retrieve_, \
                Traj_kin_, Traj_reach_, Traj_retrieve_, \
                time_kin_, time_reach_, time_retrieve_ = sample
            
            obj, obj_arg = DummyObject(), obj_args[obj_name]
            obj.name = obj_name
            obj.GRIP_DEPTH = obj_arg["GRIP_DEPTH"]
            obj.DIM = obj_arg["DIM"]
            obj.geometry = gscene.NAME_DICT[obj_arg["gname"]]
            obj_pscene, handles = add_object(pscene, obj)
            
            gscene.update()
#             g_checker.update_gscene(only_self_collision=True)
#             gr_checker.update_gscene(only_self_collision=True)
#             grl_checker.update_gscene(only_self_collision=True)
                            
            pscene.set_object_state(from_state)

            with gtimer.block("Grasp"):
                success_g = \
                    g_checker.plan_transition(from_state=from_state, to_state=to_state, redundancy_dict=redundancy_dict, 
                                               test_filters_only=True, timeout=TIMEOUT_SELF)

            with gtimer.block("GraspReach"):
                success_gr = \
                    gr_checker.plan_transition(from_state=from_state, to_state=to_state, redundancy_dict=redundancy_dict, 
                                               test_filters_only=True, timeout=TIMEOUT_SELF)

            with gtimer.block("GRL"):
                success_grl = \
                    grl_checker.plan_transition(from_state=from_state, to_state=to_state, redundancy_dict=redundancy_dict, 
                                               test_filters_only=True, timeout=TIMEOUT_SELF)

            labels.append(success_retrieve_)
            g_results.append(success_g)
            gr_results.append(success_gr)
            grl_results.append(success_grl)
                
            pscene.set_object_state(from_state)
            pscene.remove_subject(obj_pscene.oname)
            for handle in handles:
                gscene.remove(handle)  
            gscene.update_markers_all()  
            N_tot += 1
            N_suc += success_retrieve_
            if N_tot>=N_MAX:
                break
        print("GRL: {} / SUCCESS: {} / TOTAL: {} / ELAPSED TIME: {} s / ETA: {} s".format(
            np.sum(grl_results), N_suc, N_tot, *np.round(np.array(gtimer.eta("full_loop", N_tot, N_MAX))/1000, 1)))
        if N_tot>=N_MAX:
            break
    if N_tot>=N_MAX:
        break

DATASET_LIST: ['20210611-142747', '20210611-143200', '20210611-143507', '20210611-143922', '20210611-144222', '20210611-144701', '20210611-145001', '20210611-145428', '20210611-145955', '20210611-150331']
DATASET_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-142747
DATA_LIST: ['00001.pkl', '00002.pkl', '00003.pkl', '00004.pkl', '00005.pkl', '00006.pkl', '00007.pkl', '00008.pkl', '00009.pkl', '00010.pkl', '00011.pkl', '00012.pkl', '00013.pkl', '00014.pkl', '00015.pkl', '00016.pkl', '00017.pkl', '00018.pkl', '00019.pkl', '00020.pkl']
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-142747/00001.pkl
GRL: 0 / SUCCESS: 0 / TOTAL: 3 / ELAPSED TIME: 0.0 s / ETA: 6.8 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-142747/00002.pkl
GRL: 0 / SUCCESS: 0 / TOTAL: 9 / ELAPSED TIME: 0.1 s / ETA: 9.7 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-142747/00003.pkl
GRL: 0 / SUCCESS: 0 / TOTAL: 15 / ELAP

GRL: 33 / SUCCESS: 31 / TOTAL: 222 / ELAPSED TIME: 3.7 s / ETA: 16.6 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-143507/00005.pkl
GRL: 33 / SUCCESS: 31 / TOTAL: 225 / ELAPSED TIME: 3.7 s / ETA: 16.5 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-143507/00006.pkl
GRL: 34 / SUCCESS: 32 / TOTAL: 228 / ELAPSED TIME: 3.8 s / ETA: 16.5 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-143507/00007.pkl
GRL: 34 / SUCCESS: 32 / TOTAL: 231 / ELAPSED TIME: 3.8 s / ETA: 16.5 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-143507/00008.pkl
GRL: 34 / SUCCESS: 32 / TOTAL: 237 / ELAPSED TIME: 3.9 s / ETA: 16.2 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-143507/00009.pkl
GRL: 39 / SUCCESS: 37 / TOTAL: 243 / ELAPSED TIME: 4.2 s / ETA: 17.1 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-143507/00010.pkl
GRL: 39 / SUCCESS: 37 / TOTAL: 252

GRL: 64 / SUCCESS: 58 / TOTAL: 462 / ELAPSED TIME: 7.0 s / ETA: 15.2 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-144222/00013.pkl
GRL: 64 / SUCCESS: 58 / TOTAL: 468 / ELAPSED TIME: 7.1 s / ETA: 15.1 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-144222/00014.pkl
GRL: 64 / SUCCESS: 58 / TOTAL: 471 / ELAPSED TIME: 7.1 s / ETA: 15.0 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-144222/00015.pkl
GRL: 64 / SUCCESS: 58 / TOTAL: 477 / ELAPSED TIME: 7.1 s / ETA: 14.9 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-144222/00016.pkl
GRL: 64 / SUCCESS: 58 / TOTAL: 480 / ELAPSED TIME: 7.1 s / ETA: 14.9 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-144222/00017.pkl
GRL: 64 / SUCCESS: 58 / TOTAL: 486 / ELAPSED TIME: 7.2 s / ETA: 14.8 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-144222/00018.pkl
GRL: 64 / SUCCESS: 58 / TOTAL: 489

GRL: 101 / SUCCESS: 89 / TOTAL: 702 / ELAPSED TIME: 10.9 s / ETA: 15.5 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-145001/00018.pkl
GRL: 101 / SUCCESS: 89 / TOTAL: 708 / ELAPSED TIME: 11.0 s / ETA: 15.5 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-145001/00019.pkl
GRL: 101 / SUCCESS: 89 / TOTAL: 714 / ELAPSED TIME: 11.0 s / ETA: 15.4 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-145001/00020.pkl
GRL: 103 / SUCCESS: 91 / TOTAL: 723 / ELAPSED TIME: 11.2 s / ETA: 15.5 s
DATASET_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-145428
DATA_LIST: ['00001.pkl', '00002.pkl', '00003.pkl', '00004.pkl', '00005.pkl', '00006.pkl', '00007.pkl', '00008.pkl', '00009.pkl', '00010.pkl', '00011.pkl', '00012.pkl', '00013.pkl', '00014.pkl', '00015.pkl', '00016.pkl', '00017.pkl', '00018.pkl', '00019.pkl']
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-145428/00001.pkl
GRL: 1

GRL: 141 / SUCCESS: 119 / TOTAL: 951 / ELAPSED TIME: 15.1 s / ETA: 15.9 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-150331/00004.pkl
GRL: 143 / SUCCESS: 122 / TOTAL: 954 / ELAPSED TIME: 15.3 s / ETA: 16.0 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-150331/00005.pkl
GRL: 144 / SUCCESS: 123 / TOTAL: 960 / ELAPSED TIME: 15.4 s / ETA: 16.1 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-150331/00006.pkl
GRL: 144 / SUCCESS: 123 / TOTAL: 963 / ELAPSED TIME: 15.6 s / ETA: 16.2 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-150331/00007.pkl
GRL: 144 / SUCCESS: 123 / TOTAL: 966 / ELAPSED TIME: 15.6 s / ETA: 16.2 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-150331/00008.pkl
GRL: 144 / SUCCESS: 123 / TOTAL: 969 / ELAPSED TIME: 15.7 s / ETA: 16.2 s
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-hoof/panda/20210611-150331/00009.pkl
GRL: 146 / SUCCE

In [15]:
print(gtimer)
print("")
print("=========== GraspChecker ==============")
print_confusion_mat(GT=labels, Res=g_results)
print("")
print("=========== GraspReacher ==============")
print_confusion_mat(GT=labels, Res=gr_results)
print("")
print("=========== TotalChecker ==============")
print_confusion_mat(GT=labels, Res=grl_results)

full_loop: 	1548893.5 ms/200 = 7744.5 ms (20.464/16322.285)
Grasp: 	2218.4 ms/1000 = 2.2 ms (0.972/7.676)
GraspReach: 	2433.6 ms/1000 = 2.4 ms (1.075/8.063)
GRL: 	9532.3 ms/1000 = 9.5 ms (1.059/84.022)


=========== GraspChecker ==============
	 PP 	 	 PN 	 	 1000
GP 	 129 	 	 0 	 	 100.00%
GN 	 506 	 	 365 	 41.91%
AL 	 20.31% 	 100.00% 	 49.40%

=========== GraspReacher ==============
	 PP 	 	 PN 	 	 1000
GP 	 127 	 	 2 	 	 98.45%
GN 	 33 	 	 838 	 96.21%
AL 	 79.38% 	 99.76% 	 96.50%

=========== TotalChecker ==============
	 PP 	 	 PN 	 	 1000
GP 	 125 	 	 4 	 	 96.90%
GN 	 25 	 	 846 	 97.13%
AL 	 83.33% 	 99.53% 	 97.10%


# Panda
```
=========== GraspChecker ==============
	 PP 	 	 PN 	 	 1000
GP 	 129 	 	 0 	 	 100.00%
GN 	 506 	 	 365 	 41.91%
AL 	 20.31% 	 100.00% 	 49.40%

=========== GraspReacher ==============
	 PP 	 	 PN 	 	 1000
GP 	 127 	 	 2 	 	 98.45%
GN 	 33 	 	 838 	 96.21%
AL 	 79.38% 	 99.76% 	 96.50%

=========== TotalChecker ==============
	 PP 	 	 PN 	 	 1000
GP 	 125 	 	 4 	 	 96.90%
GN 	 25 	 	 846 	 97.13%
AL 	 83.33% 	 99.53% 	 97.10%
```